In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utils_alki import build_df_from_alki_csv, build_ts_from_alki_df

data_alki_df = build_df_from_alki_csv()
series_alki = build_ts_from_alki_df(data_alki_df)

len(series_alki)

In [ ]:
CUSTOMER = "NUTRAVANCE"

series_customer = series_alki[CUSTOMER]

In [ ]:
import numpy as np

series_na_mask = series_customer.pd_series().isna()
sample_weight = np.ones((len(series_customer), 1))
sample_weight[series_na_mask, 0] = 0.2
sample_weight = series_customer.with_values(sample_weight)

In [ ]:
from darts.dataprocessing.transformers.missing_values_filler import MissingValuesFiller

transformer_filler = MissingValuesFiller()
series_filled = transformer_filler.transform(series_customer)

series_filled.plot()

In [ ]:
training_cuttoff = 0.8

series_train, series_val = series_filled.split_after(training_cuttoff)

In [ ]:
from darts.dataprocessing.transformers import Scaler

transformer_scaler = Scaler()

series_train_t = transformer_scaler.fit_transform(series_train)
series_val_t = transformer_scaler.fit_transform(series_val)
series_t = transformer_scaler.fit_transform(series_customer)

In [ ]:
from darts.models import TFTModel
from darts.utils.likelihood_models import QuantileRegression


# default quantiles for QuantileRegression
quantiles = [
    0.01,
    0.05,
    0.1,
    0.15,
    0.2,
    0.25,
    0.3,
    0.4,
    0.5,
    0.6,
    0.7,
    0.75,
    0.8,
    0.85,
    0.9,
    0.95,
    0.99,
]
input_chunk_length = 24
forecast_horizon = 12

add_encoders = {
    "cyclic": {"future": ["month"]},
    "datetime_attribute": {"future": ["dayofweek"]},
    "transformer": Scaler(),
}

my_model = TFTModel(
    input_chunk_length=input_chunk_length,
    output_chunk_length=forecast_horizon,
    hidden_size=64,
    lstm_layers=1,
    num_attention_heads=4,
    full_attention=True,
    dropout=0.1,
    batch_size=512,
    n_epochs=600,
    add_relative_index=False,
    add_encoders=add_encoders,
    likelihood=QuantileRegression(
        quantiles=quantiles
    ),  # QuantileRegression is set per default
    # loss_fn=MSELoss(),
    random_state=42,
    log_tensorboard=True,
)
my_model

In [ ]:
my_model.fit(
    series_train_t, val_series=series_val_t, verbose=True, sample_weight=sample_weight
)

In [ ]:
# before starting, we define some constants
num_samples = 200

figsize = (16, 6)
lowest_q, low_q, high_q, highest_q = 0.01, 0.1, 0.9, 0.99
label_q_outer = f"{int(lowest_q * 100)}-{int(highest_q * 100)}th percentiles"
label_q_inner = f"{int(low_q * 100)}-{int(high_q * 100)}th percentiles"

In [ ]:
from darts.metrics import mape, mase, rmse
import matplotlib.pyplot as plt


def eval_model(model, n, actual_series, val_series):
    cov_pred_past, cov_pred_future = my_model.generate_predict_encodings(
        n, series_train_t
    )

    pred_series = model.predict(
        n=n,
        past_covariates=cov_pred_past,
        future_covariates=cov_pred_future,
        num_samples=num_samples,
    )

    # plot actual series
    plt.figure(figsize=figsize)
    actual_series[val_series.start_time() : val_series.end_time()].plot(label="actual")

    # plot prediction with quantile ranges
    pred_series.plot(
        low_quantile=lowest_q, high_quantile=highest_q, label=label_q_outer
    )
    pred_series.plot(low_quantile=low_q, high_quantile=high_q, label=label_q_inner)

    plt.title("MAPE: {:.2f}%".format(rmse(val_series, pred_series)))
    plt.legend()


n = (series_val_t.end_time() - series_val_t.start_time()).days

eval_model(my_model, n, series_t, series_val_t)